In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0413 00:55:02.396085 140633614988672 deprecation.py:323] From <ipython-input-1-2f7a547f03eb>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0413 00:55:02.400234 140633614988672 deprecation.py:323] From /home/george/anaconda2/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0413 00:55:02.422096 140633614988672 deprecation.py:323] From /home/george/anaconda2/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future ve

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0413 00:55:12.919698 140633614988672 deprecation.py:323] From /home/george/anaconda2/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0413 00:55:12.931915 140633614988672 deprecation.py:323] From /home/george/anaconda2/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0413 00:55:15.772166 140633614988672 deprecation.py:323] From /home/george/anaconda2/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#parâmetros gerais
learning_rate = 0.001
epochs = 10
batch_size = 200

In [4]:
#parametros da rede neural
n_entrada = 784
n_classes = 10

In [5]:
#variáveis preditoras e target (em forma de placeholders)
x = tf.placeholder(tf.float32, [None, n_entrada])
y = tf.placeholder(tf.float32, [None, n_classes])

#criando funções com os procedimentos da CNN (convolução e maxpooling)...

In [12]:
def conv2d(entrada, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(entrada, w, strides=[1,1,1,1], padding = "VALID"), b))

In [13]:
def max_pool(entrada, k):
    return tf.nn.max_pool(entrada, ksize = [1,k,k,1], strides = [1,k,k,1], padding="VALID")

#Normalizando a entrada para a convolução conseguir ser executada

In [14]:
x_redimensionado = tf.reshape(x, shape = [-1,28,28,1])

#Iniciando os pesos e bias

In [15]:
w1=tf.Variable(tf.random_normal([5,5,1,32], stddev=0.05))
w2=tf.Variable(tf.random_normal([5,5,32,64], stddev=0.05))
w_denso_1 = tf.Variable(tf.random_normal([4*4*64,80], stddev=0.0063))
w_out = tf.Variable(tf.random_normal([80, n_classes], stddev=0.04))

b1 = tf.Variable(tf.zeros([32]))
b2 = tf.Variable(tf.zeros([64]))
b_denso_1 = tf.Variable(tf.zeros([80]))
b_out = tf.Variable(tf.zeros([n_classes]))

#definindo a estrutura da CNN

In [18]:
#camada 1
conv1 = conv2d(x_redimensionado,w1,b1)
pool1 = max_pool(conv1, k=2)
#drop1 = tf.nn.dropout(pool1, rate=0.2)

#camada 2
conv2 = conv2d(pool1,w2,b2)
pool2 = max_pool(conv2, k=2)
#drop2 = tf.nn.dropout(pool2, rate=0.2)

#camada densa oculta
drop2_redimensionada = tf.reshape(pool2, shape=[-1, w_denso_1.get_shape().as_list()[0]]) #flatten
densa = tf.nn.relu(tf.add(tf.matmul(drop2_redimensionada, w_denso_1), b_denso_1))
drop_densa = tf.nn.dropout(densa, rate=0.2)

#camada de saída
out = tf.add(tf.matmul(drop_densa, w_out), b_out)

#otimizador e função de custo

In [20]:
custo = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=y))
otimizador = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(custo)

#avaliando o modelo

In [22]:
acertos = tf.equal(tf.argmax(out,1), tf.argmax(y,1))#checa quantos elementos são iguais
acuracia = tf.reduce_mean(tf.cast(acertos, tf.float32))

#inicializando as variáveis

In [24]:
init = tf.global_variables_initializer()

#abrindo a sessão
with tf.Session() as sess:
    sess.run(init)
    #cliclo de treinamento - epochs
    for epoch in range(epochs):
        custo_medio = 0.0
        total_batches = int(mnist.train.num_examples / batch_size)
        
        #loop para os batchs
        for i in range(total_batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            #fit training usando batch data
            sess.run(otimizador, feed_dict={x: batch_x, y: batch_y})
            
            #computando o custo médio de uma epoch completa: Soma tudo e divide pela quantidade de batchs
            custo_medio += sess.run(custo, feed_dict={x: batch_x, y: batch_y}) / total_batches
            
        #calculando a acurácia em cada época
        acuracia_teste = sess.run(acuracia, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        
        #mostrando resultados após cada época
        print("Epoch: ", "{},".format((epoch + 1)), "Custo medio de treino = ", "{:.3f}". format(custo_medio))
        print("Acc test = ", "{:.3f}".format(acuracia_teste))
        
    print ("Treinamento concluído!")
    print("Acc do Modelo:", acuracia.eval({x: mnist.test.images, y: mnist.test.labels}))

('Epoch: ', '1,', 'Custo medio de treino = ', '0.368')
('Acc test = ', '0.966')
('Epoch: ', '2,', 'Custo medio de treino = ', '0.094')
('Acc test = ', '0.980')
('Epoch: ', '3,', 'Custo medio de treino = ', '0.066')
('Acc test = ', '0.983')
('Epoch: ', '4,', 'Custo medio de treino = ', '0.050')
('Acc test = ', '0.986')
('Epoch: ', '5,', 'Custo medio de treino = ', '0.040')
('Acc test = ', '0.989')
('Epoch: ', '6,', 'Custo medio de treino = ', '0.035')
('Acc test = ', '0.988')
('Epoch: ', '7,', 'Custo medio de treino = ', '0.029')
('Acc test = ', '0.989')
('Epoch: ', '8,', 'Custo medio de treino = ', '0.025')
('Acc test = ', '0.987')
('Epoch: ', '9,', 'Custo medio de treino = ', '0.022')
('Acc test = ', '0.988')
('Epoch: ', '10,', 'Custo medio de treino = ', '0.020')
('Acc test = ', '0.990')
Treinamento concluído!
('Acc do Modelo:', 0.9902)
